In [1]:
%pip install praw sentiment-analysis-spanish python-dotenv openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
import praw
import pandas as pd
from dotenv import load_dotenv
import os
import datetime
from sentiment_analysis_spanish import sentiment_analysis

In [3]:
# Carga las variables de entorno del archivo .env
load_dotenv()

# Accede a las variables de entorno
client_id = os.getenv('REDDIT_CLIENT_ID')
client_secret = os.getenv('REDDIT_CLIENT_SECRET')
user_agent = os.getenv('REDDIT_USER_AGENT')


In [4]:
# https://www.reddit.com/prefs/apps
reddit = praw.Reddit(
   client_id= client_id , # personal use script
   client_secret= client_secret ,# secret
   user_agent= user_agent  # my_user_agent
)

In [5]:
# Para los comentarios:
# Crea una instancia del analizador de sentimientos
sentiment = sentiment_analysis.SentimentAnalysisSpanish()
hot_posts = reddit.subreddit('chile').hot(limit=450)
data = []
count = 0

for post in hot_posts:
    if count >= 450:
        break

    # Extrae los comentarios del post
    post.comments.replace_more(limit=None)
    for comment in post.comments.list():
        if count >= 450:
            break

        # Analiza el sentimiento del comentario
        if comment.body:
            analysis_comment = sentiment.sentiment(comment.body)
            if analysis_comment > 0.05:
                sentiment_label_comment = 'Positivo'
            elif analysis_comment < -0.05:
                sentiment_label_comment = 'Negativo'
            else:
                sentiment_label_comment = 'Neutral'

            # Guarda los resultados en una lista
            data.append({
                'Texto': comment.body,
                'Fecha': datetime.datetime.fromtimestamp(comment.created).strftime('%d/%m/%Y'),
                'Nombre_Usuario': comment.author.name if comment.author else '',
                'Concepto_asociado': sentiment_label_comment,
                'Fuente': f'https://www.reddit.com{comment.permalink}',
                'Género': "S/N",
                'Grupo_Étareo': "S/N",
                'Escolaridad': "S/N"
            })

            count += 1

# Crea un DataFrame con los datos
df = pd.DataFrame(data)

# Guarda los datos en un archivo CSV
#df.to_csv('reddit_comments.csv', index=False)

# Guarda el dataframe en un archivo Excel
df.to_excel('reddit_comments.xlsx', index=False)

c:\Users\ridds\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator CountVectorizer from version 0.23.2 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\ridds\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator MultinomialNB from version 0.23.2 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
# Para las publicaciones de Reddit
# Crea una instancia del analizador de sentimientos
sentiment = sentiment_analysis.SentimentAnalysisSpanish()

# Descarga los posts más populares del subreddit 'chile'
hot_posts = reddit.subreddit('chile').hot(limit=450)

def extract_data(posts):
    for post in posts:
        # Analiza el sentimiento del título del post
        if post.title:
            analysis = sentiment.sentiment(post.title)
            if analysis > 0.05:
                sentiment_label = 'Positivo'
            elif analysis < -0.05:
                sentiment_label = 'Negativo'
            else:
                sentiment_label = 'Neutral'

            # Guarda los resultados en un diccionario
            yield {
                'Texto': post.title,
                'Fecha': datetime.datetime.fromtimestamp(post.created),
                'Nombre Usuario': post.author.name if post.author else '',
                'Fuente': f'https://www.reddit.com{post.permalink}',
                'Concepto asociado': sentiment_label,
                'Género': "S/N",
                'Grupo_Étareo': "S/N",
                'Escolaridad': "S/N"
            }

# Crea un DataFrame con los datos
df = pd.DataFrame(extract_data(hot_posts))

# Guarda los datos en un archivo CSV
#df.to_csv('reddit_post.csv', index=False)

# Guarda el dataframe en un archivo Excel
df.to_excel('reddit_post.xlsx', index=False)